In [ ]:
# Import required libraries
from matplotlib import pyplot as plt
import os
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

# Initialize empty list to store GeoDataFrames
gdfs = []

# Set path to directory containing storm surge KML files
folder_path = f"./Miami/Surge data"

# Iterate through all KML files in the specified folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(".kml"):
        # Extract advisory number from filename (e.g., "036" from "AL112017_WatchWarningSS_036adv.kml")
        advisory_str = filename[-10:-7]
        advisory = int(advisory_str)

        # Create full file path
        file_path = os.path.join(folder_path, filename)

        # Read KML file into GeoDataFrame and add advisory number as a column
        gdf = gpd.read_file(file_path, driver='KML')
        gdf["advisory"] = advisory
        gdfs.append(gdf)

# Combine all individual GeoDataFrames into a single DataFrame
combined_gdf = pd.concat(gdfs, ignore_index=True)
# Convert coordinate reference system to Web Mercator (EPSG:3857)
combined_gdf = combined_gdf.to_crs("3857")

In [ ]:
# Load Miami-Dade County geographic data
a = gpd.read_file(f"../ACSData/['Florida']['Miami-Dade County']/data.gpkg", layer='Data')


# Function to convert advisory numbers to timestamps (6-hour intervals)
def convert_advisory_numbers(value):
    return 108 + ((value - 33) * 6)


# Convert advisory numbers in the combined GeoDataFrame
combined_gdf['advisory'] = combined_gdf['advisory'].apply(convert_advisory_numbers)

# Get unique timesteps where we have storm surge data
steps_with_data = set(combined_gdf["advisory"])

# Initialize list to store storm surge values for each census tract
tract_list = []

# Iterate through each census tract geometry
for tract in a["geometry"]:
    value_list = []
    # Create timeline of 216 hours (9 days) with 6-hour intervals
    for i in range(216):
        if i % 6 == 0:  # Check every 6-hour interval
            if i in steps_with_data:
                # Get storm surge data for current timestep
                step_gdf = combined_gdf[combined_gdf["advisory"] == i]
                value = 0
                check = False

                # Check if tract is within any storm surge polygons
                for index, poly in step_gdf.iterrows():
                    if tract.within(poly["geometry"]):
                        # Assign values based on storm surge status
                        if poly["Description"] == "Storm Surge Watch in Effect":
                            value_list.append(0.5)  # Watch = 0.5
                            check = True
                        else:
                            if poly["Description"] == "Storm Surge Warning in Effect":
                                value_list.append(1)  # Warning = 1.0
                                check = True

                # If tract is not in any storm surge polygon, assign 0
                if check == False:
                    value_list.append(0)
            else:
                # No storm surge data for this timestep
                value_list.append(0)
    tract_list.append(value_list)
a["storm_Surge"]=tract_list
a.to_file(f"../ACSDATA/['Florida']['Miami-Dade County']/data.gpkg", driver='GPKG', layer='Data')

In [ ]:
# Import contextily for adding basemap
import contextily as ctx

# Create visualization of storm surge data
# Set up the figure with large size for detailed view
fig, ax = plt.subplots(figsize=(15, 15))

# Filter data for advisory 40 and plot storm surge polygons
# Convert to Web Mercator projection (EPSG:3857) for proper overlay with basemap
step_gdf[step_gdf["advisory"] == 40].to_crs(epsg=3857).plot(
    ax=ax,
    column='Description',  # Color polygons based on surge status
    legend=True,  # Show legend
    categorical=True,  # Treat Description as categorical data
    legend_kwds={'title': 'Storm Surge Status'},  # Add legend title
    cmap='Set2',  # Use Set2 colormap for distinct categories
    alpha=0.7  # Set transparency for better basemap visibility
)

# Add OpenStreetMap as basemap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Set title and remove axes for cleaner map view
plt.title('Storm Surge Data for Advisory 105')
ax.set_axis_off()
plt.show()
